In [2]:
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import torch
from torchvision import transforms

from torch import nn

from PIL import Image

import os, random

In [3]:
# model load
CLASS_NUM = 40
model = torch.load('../model_save/train_2/epoch_0_val_0.69.pt').to('cpu')
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
classes = {'가는장구채': 0, '가시박': 1, '가지': 2, '개여뀌': 3, '개오동': 4, '개옻나무': 5, '곰취': 6,\
    '까마중': 7, '꽃개오동': 8, '능소화': 9, '닥풀': 10, '담배풀': 11, '대청부채': 12, '도깨비가지': 13,\
    '독미나리': 14, '맑은대쑥': 15, '묏미나리': 16, '미국까마중': 17, '미국능소화': 18, '미나리': 19,\
    '범부채': 20, '분홍장구채': 21, '붉나무': 22, '술패랭이꽃': 23, '아까시나무': 24, '어저귀': 25,\
    '여뀌': 26, '여우오줌': 27, '왕자귀나무': 28, '자귀나무': 29, '장구채': 30, '제비쑥': 31,\
    '좀담배풀': 32, '진득찰': 33, '쪽': 34, '참취': 35, '털진득찰': 36, '패랭이꽃': 37, '하늘타리': 38, '회화나무': 39}
classes_dict_inverse = {v:k for k, v in classes.items()}
poison_index = [1, 5, 7, 9, 10, 12, 13, 14, 17, 18, 20, 26, 27, 32, 35]

# 이미지 transform
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# Web
app = Flask(__name__)
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def image_predict():
    if request.method == 'POST':
        f = request.files['file']
        file_name = secure_filename(f.filename)
        img = Image.open(f)
        
        img.save('./static/image/uploads/' + file_name)
        img = Image.open('./static/image/uploads/' + file_name)
        img = transform(img)

        # 이미지의 클래스 예측
        imgs = img.unsqueeze(0) # batch index 넣어줌
        predicted = model(imgs)
        predicted = model.CF_fc1(predicted)
        predicted = model.CF_fc2(predicted)
        predicted_class_num = torch.argmax(predicted)
        pred_name = classes_dict_inverse[predicted_class_num.item()]
        pred_pro = f"%0.4f" % predicted[0][predicted_class_num]
        
        # 확인용 터미널 출력
        print(f"예측 결과는? {predicted}")
        print(f"예측은 class 번호는 ? {predicted_class_num} {pred_name}")

        # 예시 이미지 선택
        ex_num = 3
        ex_list = os.listdir('static/image/examples/' + pred_name)
        ex_idx = random.sample(range(len(ex_list)), ex_num) # 3개 예시 뽑음
        
        ex_file_name = []
        for x in ex_idx:
            ex_file_name.append(ex_list[x])
        
    return render_template('result.html', 
    upload_image='image/uploads/' + file_name, 
    example_image=ex_file_name, #'image/examples/' + pred_name + '/' + ex_file_name,
    result_pro = pred_pro,
    result_name = pred_name,
    poison_info = True if predicted_class_num.item() in poison_index else False,
    file_name = f.filename)

if __name__ == '__main__':
    app.run(port=2008, debug=True, use_reloader=False)
    #app.run(port=2033, debug=True)